In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [35]:
# Pandas ayarları: Tüm sütunları görmek istiyorum, gizleme yapma
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [36]:
df_raw = pd.read_csv("turkey_used_cars.csv")
df = df_raw.copy()


In [37]:
print(f"Veri Seti Boyutu: {df.shape}")

Veri Seti Boyutu: (253344, 8)


In [38]:
df.head()

,brand,model,variant,year,km,color,price,city
0,audi,a1,1.4 TFSI Attraction,2011,155000,White,625000,İstanbul
1,audi,a1,1.6 TDI Ambition,2011,252000,Black,625000,Aydın
2,audi,a1,1.4 TFSI Attraction,2012,203000,Black,665000,Zonguldak
3,audi,a1,1.6 TDI Ambition,2013,270000,White,835000,İstanbul
4,audi,a1,1.6 TDI Attraction,2011,228000,Black,670000,Ankara


In [39]:
print(f"Sütunlar: {df.columns}")

Sütunlar: Index(['brand', 'model', 'variant', 'year', 'km', 'color', 'price', 'city'], dtype='object')


In [40]:
duplicate_count = df.duplicated().sum()
print(f"Tekrarlayan satır sayısı: {duplicate_count}")

Tekrarlayan satır sayısı: 288


In [41]:
if duplicate_count > 0:
    df = df.drop_duplicates()
    print(f"Tekrarlayan satırlar kaldırıldı.")
    print(f"Yeni Veri Seti Boyutu: {df.shape}")

Tekrarlayan satırlar kaldırıldı.
Yeni Veri Seti Boyutu: (253056, 8)


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 253056 entries, 0 to 253343
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   brand    253056 non-null  object
 1   model    253056 non-null  object
 2   variant  253056 non-null  object
 3   year     253056 non-null  int64 
 4   km       253056 non-null  int64 
 5   color    253056 non-null  object
 6   price    253056 non-null  int64 
 7   city     253056 non-null  object
dtypes: int64(3), object(5)
memory usage: 17.4+ MB


In [43]:
df.isnull().sum()

brand      0
model      0
variant    0
year       0
km         0
color      0
price      0
city       0
dtype: int64

In [44]:
df["variant"].unique()

array(['1.4 TFSI Attraction', '1.6 TDI Ambition', '1.6 TDI Attraction',
       ..., '1.5 R-Design', '1.5 Inscription', '1.5 Momentum'],
      shape=(2118,), dtype=object)

Variant Sütununu 3'e Bölüyoruz.

In [45]:
def parse_variant(text):
    # Hata önleyici: Boş veri gelirse güvenli çıkış
    if pd.isna(text):
        return pd.Series([None, 'Unknown', 'Unknown'])
    
    text = str(text).lower().strip()
    original_text = text

    # --- A. MOTOR HACMİ (Volume) ---
    # \d\.\d -> Noktayı literal olarak algılaması için kaçış karakteri ekledim
    vol_match = re.search(r'\b(\d\.\d)\b', text)
    volume = float(vol_match.group(1)) if vol_match else None

    # --- B. MOTOR TEKNOLOJİSİ (Engine Tech) ---
    engine_codes = [
        'tfsi', 'tdi', 'tsi', 'cdti', 'bluehdi', 'hdi', 'dci', 
        'vtec', 'multijet', 'hybrid', 'electric', 'gdi', 'cdi',
        'blue', 'eco', 'turbo', '4wd', 'awd'
    ]
    
    engine_type = 'Standard'
    for code in engine_codes:
        if code in text:
            engine_type = code
            break 
    # DİKKAT: Döngü burada bitti. Aşağıdaki kodlar artık döngünün DIŞINDA olmalı.

    # --- C. DONANIM PAKETİ (Package) ---
    package = original_text

    if volume:
        package = package.replace(str(volume), '')

    if engine_type != 'Standard':
        package = package.replace(engine_type, '')

    # Temizlik işlemleri
    package = package.strip().replace('-', ' ')
    package = re.sub(r'\s+', ' ', package)

    if not package or len(package) < 2:
        package = 'Base'
    
    # Return komutu fonksiyonun en sonunda, en sola yaslı (1 tab içeride) olmalı
    return pd.Series([volume, engine_type, package])

# 🚀 UYGULAMA (Garanti Yöntem - Adım Adım)
print("1. Variant parçalanıyor (İşlem biraz sürebilir)...")
# apply işlemi sonucunu geçici bir değişkene atıyoruz
temp_df = df['variant'].apply(parse_variant)

# 2. Sütun isimlerini netleştiriyoruz (0, 1, 2 yerine anlamlı isimler)
temp_df.columns = ['engine_volume', 'engine_tech', 'package']

# 3. Ana DataFrame'e monte ediyoruz (En güvenli yol)
df['engine_volume'] = temp_df['engine_volume']
df['engine_tech'] = temp_df['engine_tech']
df['package'] = temp_df['package']

# 🏁 Sonuç Kontrolü
print("-" * 50)
print("✅ İŞLEM TAMAM. Örnek Veriler:")
print(df[['variant', 'engine_volume', 'engine_tech', 'package']].sample(15))
         

1. Variant parçalanıyor (İşlem biraz sürebilir)...
--------------------------------------------------
✅ İŞLEM TAMAM. Örnek Veriler:
                               variant  engine_volume engine_tech                  package
231821  1.6 TDI BlueMotion Comfortline            1.6         tdi   bluemotion comfortline
182426             1.5 DCI Authentique            1.5         dci              authentique
222802                1.6 TDI Highline            1.6         tdi                 highline
40624                  1.5 dCi Stepway            1.5         dci                  stepway
39383                 1.5 BlueHDI Feel            1.5     bluehdi                     feel
237368  1.4 TSI BlueMotion Comfortline            1.4         tsi   bluemotion comfortline
58183                1.3 Multijet Easy            1.3    multijet                     easy
213107                1.5 Flame X-Pack            1.5    Standard             flame x pack
21750          418i Gran Coupe M Sport           

KALDIĞIM YER : NaN DEĞERLERİ DÜZENLE 

In [47]:
# Temizlenmiş ve feature'ları çıkarılmış veriyi kaydet
#df.to_csv('clean_data.csv', index=False)
#print("✅ clean_data.csv kaydedildi. Operasyonun 1. aşaması tamam.")